In [1]:


from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from PIL import Image
from torchvision import transforms
import random

import numpy as np
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling1D,Input,concatenate, Activation, Dropout, Lambda,Concatenate, Reshape, Dot,MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Subtract
from tensorflow.keras.layers import Input, Dot, Multiply
from tensorflow.keras.optimizers import Adam , SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

In [2]:
augmented_metadata_path = "augmented_metadata.csv"
augmented_images_dir = "augmented_dataset_ham10k"
augmented_metadata = pd.read_csv(augmented_metadata_path)

from PIL import Image
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [22]:
import pandas as pd

# Assuming 'augmented_metadata_path' is the path to your CSV file
augmented_metadata_path = 'augmented_metadata.csv'  # Replace with your actual file path

# Load the CSV into a Pandas DataFrame
metadata = pd.read_csv(augmented_metadata_path)

# Now you can inspect the DataFrame
print(metadata.head())


     lesion_id      image_id   dx dx_type   age   sex localization  \
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp   
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp   
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp   
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp   
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear   

                                image_path  
0  HAM10000_images_part_1\ISIC_0027419.jpg  
1  HAM10000_images_part_1\ISIC_0025030.jpg  
2  HAM10000_images_part_1\ISIC_0026769.jpg  
3  HAM10000_images_part_1\ISIC_0025661.jpg  
4  HAM10000_images_part_2\ISIC_0031633.jpg  


In [23]:
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,HAM10000_images_part_1\ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,HAM10000_images_part_1\ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,HAM10000_images_part_1\ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,HAM10000_images_part_1\ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,HAM10000_images_part_2\ISIC_0031633.jpg


In [9]:
def preprocess_image(image_path):
    # Load and resize image
    image = Image.open(image_path).convert("RGB")
    image = image.resize((128, 128))

    # Convert to grayscale by averaging RGB channels
    image_np = np.array(image)
    gray_image = np.mean(image_np, axis=2, keepdims=True)  # Shape: (224, 224, 1)

    # Normalize to [0, 1] range
    gray_image = gray_image / 255.0
    return gray_image

In [10]:
class HAM10000Dataset(tf.keras.utils.Sequence):
    def __init__(self, metadata, batch_size=32, is_train=True):
        self.metadata = metadata
        self.batch_size = batch_size
        self.is_train = is_train

    def __len__(self):
        return len(self.metadata) // self.batch_size

    def __getitem__(self, idx):
        # Select batch of data
        batch_metadata = self.metadata.iloc[idx * self.batch_size:(idx + 1) * self.batch_size]
        images, labels = [], []

        for _, row in batch_metadata.iterrows():
            image_path = f'augmented_dataset_ham10k/{row["image_id"]}.jpg'
            label = row['dx']

            # Skip if the image file does not exist
            if not os.path.exists(image_path):
                #print(f"Image not found: {image_path}, skipping...")
                continue

            # Process image to grayscale
            gray_image = preprocess_image(image_path)
            images.append(gray_image)
            labels.append(class_to_idx[label])

        assert len(images) == len(labels), "Mismatch between images and labels!"

        # Convert lists to numpy arrays
        images = np.array(images, dtype="float32")
        #labels = np.array(labels, dtype="int32")
        labels = tf.keras.utils.to_categorical(labels, num_classes=len(classes))

        #print(f"Batch shape: {np.array(images).shape}, Labels shape: {np.array(labels).shape}")
        return images, labels


In [24]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming `metadata` is a Pandas DataFrame containing 'image_id' and 'dx' columns.
# You can load your metadata like this:
# metadata = pd.read_csv('path_to_metadata.csv')

# Split the metadata into train and test sets
train_metadata, test_metadata = train_test_split(metadata, test_size=0.2, random_state=42, stratify=metadata['dx'])

# Define constants
BATCH_SIZE = 32

# Create dataset objects for train and test
train_dataset = HAM10000Dataset(train_metadata, batch_size=BATCH_SIZE, is_train=True)
test_dataset = HAM10000Dataset(test_metadata, batch_size=BATCH_SIZE, is_train=False)

# Helper function to load all data from a dataset
def load_data(dataset):
    images, labels = [], []
    for i in range(len(dataset)):
        batch_images, batch_labels = dataset[i]
        images.append(batch_images)
        labels.append(batch_labels)
    return np.concatenate(images), np.concatenate(labels)

# Load train and test data
train_images, train_labels = load_data(train_dataset)
test_images, test_labels = load_data(test_dataset)

# Print shapes for verification
print(f"Train images shape: {train_images.shape}, Train labels shape: {train_labels.shape}")
print(f"Test images shape: {test_images.shape}, Test labels shape: {test_labels.shape}")


Train images shape: (16588, 128, 128, 1), Train labels shape: (16588, 7)
Test images shape: (4117, 128, 128, 1), Test labels shape: (4117, 7)


In [11]:
classes = augmented_metadata['dx'].unique()
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
# Split the dataset
train_metadata, test_metadata = train_test_split(augmented_metadata, test_size=0.2, stratify=augmented_metadata['dx'])

# Initialize train and test datasets
train_dataset = HAM10000Dataset(train_metadata, batch_size=20)[0]
test_dataset = HAM10000Dataset(test_metadata, batch_size=20)[0]
#train_dataset = train_dataset.batch(32, drop_remainder=True)
train_label = HAM10000Dataset(train_metadata, batch_size=20)
test_label = HAM10000Dataset(test_metadata, batch_size=20)
# Verify shapes of a batch
train_images, train_labels = train_dataset[0]
print("Batch of images shape:", train_images.shape)  # Should be (32, 224, 224, 1)
print("Batch of labels:", train_labels.shape)

def generate_gaussian_kernel(sigma, kernel_size):
    """Generate a 2D Gaussian kernel array."""
    # Ensure that the axis values are floats to avoid type issues during operations
    ax = tf.range(-kernel_size // 2 + 1, kernel_size // 2 + 1, dtype=tf.float32)
    xx, yy = tf.meshgrid(ax, ax)
    kernel = tf.exp(-(xx**2 + yy**2) / (2. * sigma**2))
    kernel = kernel / tf.reduce_sum(kernel)
    return kernel.numpy()


Batch of images shape: (13, 128, 128, 1)
Batch of labels: (13, 7)


In [12]:
# Updated function to extract SIFT features with consistent dimensions
def extract_sift_features(images, max_features=500):
    sift_features = []
    for img in images:
        keypoints, descriptors = sift.detectAndCompute((img * 255).astype('uint8'), None)
        if descriptors is None:  # Handle cases where no keypoints are detected
            descriptors = np.zeros((1, 128))
        # Flatten descriptors and pad/truncate to max_features length
        flat_descriptors = descriptors.flatten()
        if len(flat_descriptors) < max_features:
            flat_descriptors = np.pad(flat_descriptors, (0, max_features - len(flat_descriptors)))
        else:
            flat_descriptors = flat_descriptors[:max_features]
        sift_features.append(flat_descriptors)
    return np.array(sift_features, dtype=np.float32)

In [13]:
# Updated function to extract SIFT features with consistent dimensions
def extract_sift_features(images, max_features=500):
    sift_features = []
    for img in images:
        keypoints, descriptors = sift.detectAndCompute((img * 255).astype('uint8'), None)
        if descriptors is None:  # Handle cases where no keypoints are detected
            descriptors = np.zeros((1, 128))
        # Flatten descriptors and pad/truncate to max_features length
        flat_descriptors = descriptors.flatten()
        if len(flat_descriptors) < max_features:
            flat_descriptors = np.pad(flat_descriptors, (0, max_features - len(flat_descriptors)))
        else:
            flat_descriptors = flat_descriptors[:max_features]
        sift_features.append(flat_descriptors)
    return np.array(sift_features, dtype=np.float32)

In [15]:
# Initialize SIFT feature extractor
import cv2
sift = cv2.SIFT_create(nfeatures=50)

In [26]:
X_train_sift = extract_sift_features(train_images)
X_test_sift = extract_sift_features(test_images)
print(f"SIFT features extracted for training and test sets.")

SIFT features extracted for training and test sets.


In [29]:
from sklearn.svm import SVC
import numpy as np

# Convert one-hot encoded labels to class indices
train_labels_1d = np.argmax(train_labels, axis=1)
test_labels_1d = np.argmax(test_labels, axis=1)

# Train SVM on SIFT features
svm = SVC(kernel='rbf', C=1)  # Radial Basis Function kernel
svm.fit(X_train_sift, train_labels_1d)

# Evaluate SVM on test data
y_pred_svm = svm.predict(X_test_sift)

# Optionally, calculate accuracy or other metrics
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels_1d, y_pred_svm)
print(f"Accuracy: {accuracy}")


Accuracy: 0.46417294146222976


In [30]:
from sklearn.neighbors import KNeighborsClassifier

# Train KNN on SIFT features
knn = KNeighborsClassifier(n_neighbors=5)  # Using 5 nearest neighbors
knn.fit(X_train_sift, train_labels_1d)

# Evaluate KNN on test data
y_pred_knn = knn.predict(X_test_sift)
knn_accuracy = accuracy_score(test_labels_1d, y_pred_knn)

print(f"KNN Accuracy on SIFT features: {knn_accuracy:.4f}")

KNN Accuracy on SIFT features: 0.5239


In [31]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest on SIFT features
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)  # 100 trees
random_forest.fit(X_train_sift, train_labels_1d)

# Evaluate Random Forest on test data
y_pred_rf = random_forest.predict(X_test_sift)
rf_accuracy = accuracy_score(test_labels_1d, y_pred_rf)

print(f"Random Forest Accuracy on SIFT features: {rf_accuracy:.4f}")

Random Forest Accuracy on SIFT features: 0.7420


In [32]:
from sklearn.tree import DecisionTreeClassifier

# Train Decision Tree on SIFT features
decision_tree = DecisionTreeClassifier(random_state=42)  # Set a random state for reproducibility
decision_tree.fit(X_train_sift, train_labels_1d)

# Evaluate Decision Tree on test data
y_pred_dt = decision_tree.predict(X_test_sift)
dt_accuracy = accuracy_score(test_labels_1d, y_pred_dt)

print(f"Decision Tree Accuracy on SIFT features: {dt_accuracy:.4f}")

Decision Tree Accuracy on SIFT features: 0.6954


In [39]:
# Re-initialize SIFT feature extractor with 400 keypoints
sift = cv2.SIFT_create(nfeatures=400)

# Re-extract SIFT features for training and test sets
X_train_sift = extract_sift_features(train_images)
X_test_sift = extract_sift_features(test_images)

print(f"SIFT features extracted for training and test sets with 400 keypoints.")

SIFT features extracted for training and test sets with 400 keypoints.


In [40]:
# Import necessary modules
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Retrain and evaluate SVM
svm = SVC(kernel='rbf', C=1)
svm.fit(X_train_sift, train_labels_1d)
y_pred_svm = svm.predict(X_test_sift)
svm_accuracy = accuracy_score(test_labels_1d, y_pred_svm)
print(f"SVM Accuracy on SIFT features (400 keypoints): {svm_accuracy:.4f}")

# Retrain and evaluate k-NN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_sift, train_labels_1d)
y_pred_knn = knn.predict(X_test_sift)
knn_accuracy = accuracy_score(test_labels_1d, y_pred_knn)
print(f"k-NN Accuracy on SIFT features (400 keypoints): {knn_accuracy:.4f}")

# Retrain and evaluate Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train_sift, train_labels_1d)
y_pred_dt = decision_tree.predict(X_test_sift)
dt_accuracy = accuracy_score(test_labels_1d, y_pred_dt)
print(f"Decision Tree Accuracy on SIFT features (400 keypoints): {dt_accuracy:.4f}")

# Retrain and evaluate Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_sift, train_labels_1d)
y_pred_rf = random_forest.predict(X_test_sift)
rf_accuracy = accuracy_score(test_labels_1d, y_pred_rf)
print(f"Random Forest Accuracy on SIFT features (400 keypoints): {rf_accuracy:.4f}")

SVM Accuracy on SIFT features (400 keypoints): 0.4622
k-NN Accuracy on SIFT features (400 keypoints): 0.5230
Decision Tree Accuracy on SIFT features (400 keypoints): 0.7051
Random Forest Accuracy on SIFT features (400 keypoints): 0.7459


In [58]:
sift = cv2.SIFT_create(nfeatures=100)

# Re-extract SIFT features for training and test sets
X_train_sift = extract_sift_features(train_images)
X_test_sift = extract_sift_features(test_images)

print(f"SIFT features extracted for training and test sets with 100 keypoints.")

SIFT features extracted for training and test sets with 100 keypoints.


In [42]:
# Import necessary modules
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Retrain and evaluate SVM
svm = SVC(kernel='rbf', C=1)
svm.fit(X_train_sift, train_labels_1d)
y_pred_svm = svm.predict(X_test_sift)
svm_accuracy = accuracy_score(test_labels_1d, y_pred_svm)
print(f"SVM Accuracy on SIFT features (100 keypoints): {svm_accuracy:.4f}")

# Retrain and evaluate k-NN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_sift, train_labels_1d)
y_pred_knn = knn.predict(X_test_sift)
knn_accuracy = accuracy_score(test_labels_1d, y_pred_knn)
print(f"k-NN Accuracy on SIFT features (100 keypoints): {knn_accuracy:.4f}")

# Retrain and evaluate Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train_sift, train_labels_1d)
y_pred_dt = decision_tree.predict(X_test_sift)
dt_accuracy = accuracy_score(test_labels_1d, y_pred_dt)
print(f"Decision Tree Accuracy on SIFT features (100 keypoints): {dt_accuracy:.4f}")

# Retrain and evaluate Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_sift, train_labels_1d)
y_pred_rf = random_forest.predict(X_test_sift)
rf_accuracy = accuracy_score(test_labels_1d, y_pred_rf)
print(f"Random Forest Accuracy on SIFT features (100 keypoints): {rf_accuracy:.4f}")

SVM Accuracy on SIFT features (100 keypoints): 0.4586
k-NN Accuracy on SIFT features (100 keypoints): 0.5251
Decision Tree Accuracy on SIFT features (100 keypoints): 0.7059
Random Forest Accuracy on SIFT features (100 keypoints): 0.7367


In [43]:
sift = cv2.SIFT_create(nfeatures=200)

# Re-extract SIFT features for training and test sets
X_train_sift = extract_sift_features(train_images)
X_test_sift = extract_sift_features(test_images)

print(f"SIFT features extracted for training and test sets with 200 keypoints.")

SIFT features extracted for training and test sets with 200 keypoints.


In [44]:
# Import necessary modules
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Retrain and evaluate SVM
svm = SVC(kernel='rbf', C=1)
svm.fit(X_train_sift, train_labels_1d)
y_pred_svm = svm.predict(X_test_sift)
svm_accuracy = accuracy_score(test_labels_1d, y_pred_svm)
print(f"SVM Accuracy on SIFT features (200 keypoints): {svm_accuracy:.4f}")

# Retrain and evaluate k-NN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_sift, train_labels_1d)
y_pred_knn = knn.predict(X_test_sift)
knn_accuracy = accuracy_score(test_labels_1d, y_pred_knn)
print(f"k-NN Accuracy on SIFT features (200 keypoints): {knn_accuracy:.4f}")

# Retrain and evaluate Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train_sift, train_labels_1d)
y_pred_dt = decision_tree.predict(X_test_sift)
dt_accuracy = accuracy_score(test_labels_1d, y_pred_dt)
print(f"Decision Tree Accuracy on SIFT features (200 keypoints): {dt_accuracy:.4f}")

# Retrain and evaluate Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_sift, train_labels_1d)
y_pred_rf = random_forest.predict(X_test_sift)
rf_accuracy = accuracy_score(test_labels_1d, y_pred_rf)
print(f"Random Forest Accuracy on SIFT features (200 keypoints): {rf_accuracy:.4f}")

SVM Accuracy on SIFT features (200 keypoints): 0.4622
k-NN Accuracy on SIFT features (200 keypoints): 0.5227
Decision Tree Accuracy on SIFT features (200 keypoints): 0.7049
Random Forest Accuracy on SIFT features (200 keypoints): 0.7462


In [45]:
sift = cv2.SIFT_create(nfeatures=500)

# Re-extract SIFT features for training and test sets
X_train_sift = extract_sift_features(train_images)
X_test_sift = extract_sift_features(test_images)

print(f"SIFT features extracted for training and test sets with 500 keypoints.")

SIFT features extracted for training and test sets with 500 keypoints.


In [46]:
# Import necessary modules
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Retrain and evaluate SVM
svm = SVC(kernel='rbf', C=1)
svm.fit(X_train_sift, train_labels_1d)
y_pred_svm = svm.predict(X_test_sift)
svm_accuracy = accuracy_score(test_labels_1d, y_pred_svm)
print(f"SVM Accuracy on SIFT features (500 keypoints): {svm_accuracy:.4f}")

# Retrain and evaluate k-NN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_sift, train_labels_1d)
y_pred_knn = knn.predict(X_test_sift)
knn_accuracy = accuracy_score(test_labels_1d, y_pred_knn)
print(f"k-NN Accuracy on SIFT features (500 keypoints): {knn_accuracy:.4f}")

# Retrain and evaluate Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train_sift, train_labels_1d)
y_pred_dt = decision_tree.predict(X_test_sift)
dt_accuracy = accuracy_score(test_labels_1d, y_pred_dt)
print(f"Decision Tree Accuracy on SIFT features (500 keypoints): {dt_accuracy:.4f}")

# Retrain and evaluate Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_sift, train_labels_1d)
y_pred_rf = random_forest.predict(X_test_sift)
rf_accuracy = accuracy_score(test_labels_1d, y_pred_rf)
print(f"Random Forest Accuracy on SIFT features (500 keypoints): {rf_accuracy:.4f}")

SVM Accuracy on SIFT features (500 keypoints): 0.4622
k-NN Accuracy on SIFT features (500 keypoints): 0.5230
Decision Tree Accuracy on SIFT features (500 keypoints): 0.7051
Random Forest Accuracy on SIFT features (500 keypoints): 0.7459


In [59]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization

# Define the CNN model for SIFT features
def create_cnn_for_sift(input_shape, num_classes):
    model = Sequential([
        # Convolutional Layer 1
        Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),

        # Convolutional Layer 2
        Conv1D(128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),

        # Convolutional Layer 3
        Conv1D(256, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),

        # Flatten and Fully Connected Layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')  # Output layer for classification
    ])
    
    return model

# Example usage:
# Assuming SIFT features have 128 dimensions per feature vector
input_shape = (100, 1)  # For SIFT features (e.g., 128 descriptors)
num_classes = 10  # Number of classes (adjust as needed for your dataset)

# Create the model
model = create_cnn_for_sift(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy if labels are integers
              metrics=['accuracy'])

# Summary of the model
model.summary()


C:\Users\saadaoui\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)                    │ (None, 100, 64)             │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 100, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_6 (MaxPooling1D)       │ (None, 50, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_7 (Conv1D)                    │ (None, 50, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 50, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_7 (MaxPooling1D)       │ (None, 25, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_8 (Conv1D)                    │ (None, 25, 256)             │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 25, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_8 (MaxPooling1D)       │ (None, 12, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 3072)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 256)                 │         786,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 914,570 (3.49 MB)

 Trainable params: 913,674 (3.49 MB)

 Non-trainable params: 896 (3.50 KB)

In [60]:
model.fit(X_train_sift, train_labels_1d, epochs=20, batch_size=64, validation_split=0.2,verbose=1)

Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_8" is incompatible with the layer: expected axis -1 of input shape to have value 3072, but received input with shape (None, 15872)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 500), dtype=float32)
  • training=True
  • mask=None

In [55]:
test_loss, test_acc = model.evaluate(X_test_sift, test_labels_1d)
print(f"Test accuracy: {test_acc}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5868 - loss: 1.0226
Test accuracy: 0.5941219329833984


In [56]:
sift = cv2.SIFT_create(nfeatures=25)

# Re-extract SIFT features for training and test sets
X_train_sift = extract_sift_features(train_images)
X_test_sift = extract_sift_features(test_images)

print(f"SIFT features extracted for training and test sets with 25 keypoints.")

SIFT features extracted for training and test sets with 25 keypoints.


In [57]:
# Import necessary modules
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Retrain and evaluate SVM
svm = SVC(kernel='rbf', C=1)
svm.fit(X_train_sift, train_labels_1d)
y_pred_svm = svm.predict(X_test_sift)
svm_accuracy = accuracy_score(test_labels_1d, y_pred_svm)
print(f"SVM Accuracy on SIFT features (25 keypoints): {svm_accuracy:.4f}")

# Retrain and evaluate k-NN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_sift, train_labels_1d)
y_pred_knn = knn.predict(X_test_sift)
knn_accuracy = accuracy_score(test_labels_1d, y_pred_knn)
print(f"k-NN Accuracy on SIFT features (25 keypoints): {knn_accuracy:.4f}")

# Retrain and evaluate Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train_sift, train_labels_1d)
y_pred_dt = decision_tree.predict(X_test_sift)
dt_accuracy = accuracy_score(test_labels_1d, y_pred_dt)
print(f"Decision Tree Accuracy on SIFT features (25 keypoints): {dt_accuracy:.4f}")

# Retrain and evaluate Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_sift, train_labels_1d)
y_pred_rf = random_forest.predict(X_test_sift)
rf_accuracy = accuracy_score(test_labels_1d, y_pred_rf)
print(f"Random Forest Accuracy on SIFT features (25 keypoints): {rf_accuracy:.4f}")

SVM Accuracy on SIFT features (25 keypoints): 0.4800
k-NN Accuracy on SIFT features (25 keypoints): 0.5424
Decision Tree Accuracy on SIFT features (25 keypoints): 0.7088
Random Forest Accuracy on SIFT features (25 keypoints): 0.7542


In [7]:
def create_fixed_gaussian_layer(sigma, kernel_size, input_channels, stride):
    """Create a fixed Gaussian Conv2D layer."""
    kernel = generate_gaussian_kernel(sigma, kernel_size)
    kernel = np.expand_dims(kernel, axis=-1)  # Shape: (kernel_size, kernel_size, 1)
    kernel = np.repeat(kernel, input_channels, axis=-1)  # Shape: (kernel_size, kernel_size, input_channels)
    kernel = np.expand_dims(kernel, axis=-1)  # Shape: (kernel_size, kernel_size, input_channels, 1)

    # Use Conv2D for applying Gaussian kernel with stride
    gauss_layer = layers.Conv2D(filters=input_channels, kernel_size=(kernel_size, kernel_size),
                                strides=(stride, stride), padding="same", use_bias=False)
    gauss_layer.build((None, None, None, input_channels))  # Adjust input shape in build method
    gauss_layer.set_weights([kernel])  # Set Gaussian kernel as weights
    gauss_layer.trainable = False  # Freeze weights
    return gauss_layer

In [20]:
class KeypointDetectionLayer(layers.Layer):
    def __init__(self, contrast_threshold=0.0001, edge_threshold=1, **kwargs):
        super(KeypointDetectionLayer, self).__init__(**kwargs)
        self.contrast_threshold = contrast_threshold
        self.edge_threshold = edge_threshold

    def call(self, inputs):
        DoG_s, DoG_s1, DoG_s2 = inputs
        DoG_s = tf.clip_by_value(DoG_s, 0.0, 1.0)
        DoG_s1=tf.clip_by_value(DoG_s1, 0.0, 1.0)
        DoG_s2=tf.clip_by_value(DoG_s2, 0.0, 1.0)

        patches_s = tf.image.extract_patches(images=DoG_s, sizes=[1, 3, 3, 1], strides=[1, 1, 1, 1], rates=[1, 1, 1, 1], padding='SAME')
        patches_s1 = tf.image.extract_patches(images=DoG_s1, sizes=[1, 3, 3, 1], strides=[1, 1, 1, 1], rates=[1, 1, 1, 1], padding='SAME')
        patches_s2 = tf.image.extract_patches(images=DoG_s2, sizes=[1, 3, 3, 1], strides=[1, 1, 1, 1], rates=[1, 1, 1, 1], padding='SAME')

        combined_patches = tf.concat([patches_s, patches_s1, patches_s2], axis=-1)

        center_pixel_s1 = patches_s1[..., 4]
        max_in_neighborhood = tf.reduce_max(combined_patches, axis=-1)
        min_in_neighborhood = tf.reduce_min(combined_patches, axis=-1)
        is_max = tf.equal(center_pixel_s1, max_in_neighborhood)
        is_min = tf.equal(center_pixel_s1, min_in_neighborhood)
        keypoints = tf.logical_or(is_max, is_min)
        high_contrast = tf.abs(center_pixel_s1) > self.contrast_threshold
        keypoints = tf.cast(tf.logical_and(keypoints, high_contrast), dtype=tf.float32)

        # Edge response elimination
        dx = 0.5 * (patches_s1[..., 5] - patches_s1[..., 3])
        dy = 0.5 * (patches_s1[..., 7] - patches_s1[..., 1])
        dxx = patches_s1[..., 5] + patches_s1[..., 3] - 2 * center_pixel_s1
        dyy = patches_s1[..., 7] + patches_s1[..., 1] - 2 * center_pixel_s1
        dxy = 0.25 * (patches_s1[..., 8] + patches_s1[..., 0] - patches_s1[..., 2] - patches_s1[..., 6])

        trH = dxx + dyy
        detH = dxx * dyy - dxy * dxy
        edge_response = (trH * trH) / (detH + 1e-10)  # Add small epsilon to avoid division by zero
        is_not_edge = edge_response < ((self.edge_threshold + 1) ** 2) / self.edge_threshold
        is_not_edge = tf.cast(is_not_edge, dtype=tf.bool)
        keypoints = tf.cast(tf.logical_and(tf.cast(keypoints, dtype=tf.bool), is_not_edge), dtype=tf.float32)

        return keypoints

In [21]:
def build_model_with_individual_outputs(input_shape,channels,learning_rate,batch_size, dropout_rate,first_dense_nodes,second_dense_nodes,l2_reg):
    input_layer = layers.Input(shape=input_shape)
    x = input_layer
    outputs = []
    stride=1
    dog_outputs = []
    #### octave 1 ###
    ##### Conv layers 1 ###
    gauss_layer1 = create_fixed_gaussian_layer(1.6, 7, channels, stride=1)
    x1 = gauss_layer1(x)
    outputs.append(x)
    outputs.append(x1)
    gauss_layer2 = create_fixed_gaussian_layer(2.26, 15, channels, stride= 1)
    x2 = gauss_layer2(x)
    outputs.append(x2)
    gauss_layer3 = create_fixed_gaussian_layer(3.2,20,channels, stride=1)
    x3 = gauss_layer3(x)
    outputs.append(x3)
    gauss_layer4 = create_fixed_gaussian_layer(4.52, 28, channels, stride= 1)
    x4 = gauss_layer4(x)
    outputs.append(x4)
    #### DOG layers 1 #####
    dog_layer1_1 = layers.Subtract(name=f'dog_{1}_{1}')([x1,x2])
    #dog_layer1_11= blackwhite()(dog_layer1_1)
    outputs.append(dog_layer1_1)
    dog_layer2_1=layers.Subtract(name=f'dog_{1}_{2}')([x2,x3])
    outputs.append(dog_layer2_1)
    #dog_layer2_11= blackwhite()(dog_layer2_1)
    #outputs.append(dog_layer2_11)
    dog_layer3_1=layers.Subtract(name=f'dog_{1}_{3}')([x3,x4])
    outputs.append(dog_layer3_1)
    #dog_layer3_11= blackwhite()(dog_layer3_1)
    # outputs.append(dog_layer3_11)
    #### keypoint detection layers 1#####
    kp_layer1_1 = KeypointDetectionLayer()([dog_layer1_1, dog_layer2_1,dog_layer3_1])
    outputs.append(kp_layer1_1)

    ###### octave2 ######
    ##### Conv layers 2 ###
    Maxpool1 = MaxPooling2D((2, 2))(x3)
    outputs.append(Maxpool1)
    Maxpool2 = MaxPooling2D((2, 2))(x4)
    outputs.append(Maxpool2)
    gauss_layer3_2 = create_fixed_gaussian_layer(6.4, 39, channels, stride=2)
    x3_2 = gauss_layer3_2(x)
    outputs.append(x3_2)
    gauss_layer4_2 = create_fixed_gaussian_layer(9.05, 55, channels, stride=2)
    x4_2 = gauss_layer4_2(x)
    outputs.append(x4_2)
    #### DOG layers 2#####
    dog_layer1_2 = layers.Subtract(name=f'dog_{2}_{1}')([Maxpool1,Maxpool2])
    #dog_layer1_22= blackwhite()(dog_layer1_2)
    outputs.append(dog_layer1_2)
    dog_layer2_2=layers.Subtract(name=f'dog_{2}_{2}')([Maxpool2,x3_2])
    #dog_layer2_22= blackwhite()(dog_layer2_2)
    outputs.append(dog_layer2_2)
    dog_layer3_2=layers.Subtract(name=f'dog_{2}_{3}')([x3_2,x4_2])
    #dog_layer3_22= blackwhite()(dog_layer3_2)
    outputs.append(dog_layer3_2)

    #### keypoint detection layers 2#####
    kp_layer1_2 = KeypointDetectionLayer()([dog_layer1_2, dog_layer2_2,dog_layer3_2])
    outputs.append(kp_layer1_2)
     ###### octave3 ######
    ##### Conv layers 3 ###
    Maxpool1_3 = MaxPooling2D((2, 2))(x3_2)
    outputs.append(Maxpool1)
    Maxpool2_3 = MaxPooling2D((2, 2))(x4_2)
    outputs.append(Maxpool2)
    gauss_layer3_3 = create_fixed_gaussian_layer(12.8, 77, channels, stride=4)
    x3_3 = gauss_layer3_3(x)
    outputs.append(x3_3)
    gauss_layer4_3= create_fixed_gaussian_layer(18.1, 110, channels, stride=4)
    x4_3 = gauss_layer4_3(x)
    outputs.append(x4_3)
    #### DOG layers 2#####
    dog_layer1_3 = layers.Subtract(name=f'dog_{3}_{1}')([Maxpool1_3,Maxpool2_3])
    #dog_layer1_33= blackwhite()(dog_layer1_2)
    outputs.append(dog_layer1_3)
    dog_layer2_3=layers.Subtract(name=f'dog_{3}_{2}')([Maxpool2_3,x3_3])
    #dog_layer2_33= blackwhite()(dog_layer2_2)
    outputs.append(dog_layer2_3)
    dog_layer3_3=layers.Subtract(name=f'dog_{3}_{3}')([x3_3,x4_3])
    #dog_layer3_33= blackwhite()(dog_layer3_3)
    outputs.append(dog_layer3_3)
    kp_layer1_3 = KeypointDetectionLayer()([dog_layer1_3, dog_layer2_3,dog_layer3_3])
    outputs.append(kp_layer1_3)




    flatten1 = Flatten()(kp_layer1_1)
    flatten2 = Flatten()(kp_layer1_2)
    flatten3 = Flatten()(kp_layer1_3)
    concatenated = Concatenate()([flatten1,flatten2,flatten3])
    dense1 = Dense(first_dense_nodes, activation='relu',kernel_regularizer=l2(l2_reg))(concatenated)
    drp=Dropout(dropout_rate)(dense1)
    dense2 = Dense(second_dense_nodes, activation='relu',kernel_regularizer=l2(l2_reg))(dense1)
    drp1=Dropout(dropout_rate)(dense2)
    dense3 = Dense(128, activation='relu',kernel_regularizer=l2(l2_reg))(dense2)
    drp3=Dropout(dropout_rate)(dense3)

    output = Dense(7, activation='softmax')(dense3)

    model = models.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=SGD(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy',  tf.keras.metrics.AUC(multi_label=False),
              tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')])
    return model

In [22]:
input_shape = (224, 224, 1)
channels = 1
model = build_model_with_individual_outputs(
        input_shape, channels, learning_rate=0.01,
        batch_size=128, dropout_rate=0.0, first_dense_nodes=32,
        second_dense_nodes=256, l2_reg=1e-06
    )

In [ ]:
log_dir = os.path.join("logs", "fit10knew", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=50, restore_best_weights=True, verbose=1
    )

In [11]:
import time

In [ ]:
# Start timing
start_time = time.time()

# Train the model and save history
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=500,
    batch_size=128,
    verbose=1,
)

# End timing
end_time = time.time()


training_time = end_time - start_time
print(f"Total training time: {training_time:.2f} seconds")


training_time_minutes = training_time / 60
print(f"Total training time: {training_time_minutes:.2f} minutes")

Epoch 1/500
1228/1228 ━━━━━━━━━━━━━━━━━━━━ 994s 804ms/step - accuracy: 0.1802 - auc_2: 0.6048 - loss: 1.8646 - top_5_accuracy: 0.8572 - val_accuracy: 0.2011 - val_auc_2: 0.6207 - val_loss: 1.7967 - val_top_5_accuracy: 0.8571
Epoch 2/500
1228/1228 ━━━━━━━━━━━━━━━━━━━━ 957s 779ms/step - accuracy: 0.2033 - auc_2: 0.6206 - loss: 1.7927 - top_5_accuracy: 0.8635 - val_accuracy: 0.1917 - val_auc_2: 0.6228 - val_loss: 1.7865 - val_top_5_accuracy: 0.8552
Epoch 3/500
 533/1228 ━━━━━━━━━━━━━━━━━━━━ 7:08 616ms/step - accuracy: 0.1960 - auc_2: 0.6176 - loss: 1.7882 - top_5_accuracy: 0.8527